In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import RMSprop, Adam
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_resnet_v2 import InceptionResNetV2,preprocess_input
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.layers import Dense
from keras.models import Model
import keras


import os
print(os.listdir("../input"))

train_dir = "../input/train/"
test_dir = "../input/test/"

dataset= pd.read_csv('../input/train_labels.csv',dtype='str')

def append_ext(fn):
    return fn+".tif"
dataset["id"]=dataset["id"].apply(append_ext)

datapath='../input/'
train_path = datapath+'train'
valid_path =  datapath+'train'
test_path=datapath+'test'


In [ ]:
import keras
train_datagen=ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.5,
    height_shift_range=0.5,
    shear_range=0.5,
    zoom_range=0.5,
    horizontal_flip=True,vertical_flip=True,
    rotation_range=90,
    rescale=1./255,
    validation_split=0.3
    )
        
train_generator = train_datagen.flow_from_dataframe(
                dataframe=dataset,
                directory=train_path,
                x_col = 'id',
                y_col = 'label',
                has_ext=False,
                subset='training',
                target_size=(96, 96),
                batch_size=32,
                class_mode='binary'
                )

validation_generator = train_datagen.flow_from_dataframe(
                dataframe=dataset,
                directory=valid_path,
                x_col = 'id',
                y_col = 'label',
                has_ext=False,
                subset='validation', # This is the trick to properly separate train and validation dataset
                target_size=(96, 96),
                batch_size=32,
                shuffle=False,
                class_mode='binary'
                )


In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2,preprocess_input
nClasses=1
channels = 3
model=InceptionResNetV2(include_top=True, weights=None,input_shape=(96,96,channels)) # sans poids ImageNet
model.layers.pop() # Suppression de la couche totalement connecté fc1000

# Ajout de la nouvelle couche de prédiction
x = model.layers[-1].output
x=Dense(nClasses, activation='sigmoid', name='predictions',kernel_initializer='glorot_normal')(x)
model = Model(input=model.input,output=x)


In [ ]:
for i in range(len(model.layers)):
    model.layers[i].trainable = True


In [ ]:
#sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=Adam(0.0001), metrics=['acc'])


In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=2)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)


In [ ]:
model.fit_generator(generator=train_generator,
              steps_per_epoch=STEP_SIZE_TRAIN,
              nb_epoch=10,
              shuffle=True,verbose=1,
              callbacks=[lr_reducer, early_stop],
              validation_data=validation_generator,
              validation_steps=STEP_SIZE_VALID)


In [ ]:
import glob
import os
from skimage.io import imread
from keras.preprocessing import image
from keras.preprocessing  import image
testing_files = glob.glob(os.path.join(test_dir,'*.tif'))
TESTING_BATCH_SIZE=10
submission = pd.DataFrame()


test_files = glob.glob(os.path.join(test_dir,'*.tif'))
submission = pd.DataFrame()
file_batch = 5000
max_idx = len(test_files)
for idx in range(0, max_idx, file_batch):
    print("Indexes: %i - %i"%(idx, idx+file_batch))
    test_df = pd.DataFrame({'path': test_files[idx:idx+file_batch]})
    test_df['id'] = test_df.path.map(lambda x: x.split('/')[3].split(".")[0])
    test_df['image'] = test_df['path'].map(imread)
    K_test = np.stack(test_df["image"].values)
    K_test = keras.applications.inception_resnet_v2.preprocess_input(K_test)
    predictions = model.predict(K_test)
    test_df['label'] = predictions
    submission = pd.concat([submission, test_df[["id", "label"]]])
submission.head()


In [ ]:
submission.to_csv("submission.csv", index = False, header = True)


In [ ]:
pd.read_csv("submission.csv").head()
